In [1]:
import auxiliary as aux
import numpy as np
import pandas as pd

# from sklearn.utils import parallel_backend
# from joblib import Parallel, delayed

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

from tqdm import tqdm

## step 1: impute subset #4

In [3]:
estimator = LGBMRegressor(
    random_state=7, n_jobs=-1,
    n_estimators=250,
    # max_depth=-1,
    num_leaves=29
)
s4 = aux.ml_impute(aux.subset[4], estimator)

100%|██████████| 15/15 [05:33<00:00, 22.25s/it]


0.297277885099793


## step 2: 

## test

In [14]:
df = aux.data.copy()
df.fillna(s4, inplace=True)

nan_cols = df.columns[df.isna().any()]
one_nan_row = df.isna().sum(axis=1) == 1      # select rows with only 1 NaN
no_nan_row = ~df.isna().any(axis=1)           # select rows without NaN

imp = df.copy()
metrics = []
for col in tqdm(nan_cols):
    # exclude rows with NaN in train columns
    nan_in_train = df[one_nan_row].drop(col, axis=1).isna().any(axis=1)        
    X_test = df[one_nan_row][~nan_in_train].drop(col, axis=1)
    # # collect fit data
    # X_train = df[no_nan_row].drop(col, axis=1)
    # y_train = df.loc[no_nan_row, col]

    # using all data
    train_mask = (df.isna().sum(axis=1) < 2) & (~df[col].isna())
    X_train = df[train_mask].drop(col, axis=1)
    y_train = df.loc[train_mask, col]
    X_train.fillna(X_train.mean(), inplace=True)
    
    # fit LGBM
    estimator = LGBMRegressor(
        random_state=7, n_jobs=-1,
        n_estimators=100,
        # max_depth=-1,
        num_leaves=23
    )
    estimator.fit(X_train, y_train)
    # predict
    train_pred = estimator.predict(X_train)
    pred = estimator.predict(X_test)
    imp.loc[X_test.index, col] = pred
    # score
    metrics.append(np.sqrt(mean_squared_error(y_train, train_pred)))    # RMSE
print(f'\n{np.mean(metrics)}')
print(f'Remain NaN: {imp.isna().sum().sum()} / {aux.data.isna().sum().sum()}')

100%|██████████| 40/40 [08:25<00:00, 12.64s/it]



0.9625808519672457
Remain NaN: 372076 / 1000000


In [ ]:
def groupstat(df, *, gcol, func='mean'):
    stats = df.groupby(gcol).transform(func)
    if stats.isna().any().any():
        stats.fillna(stats.agg(func), inplace=True)
        print(f'Stats contain NaNs! Filled with {func}')
    imp = df.fillna(stats)
    # if imp.isna().any().any():
    #     imp.fillna(imp.agg(func), inplace=True)
    #     print('Data still contain NaNs! Select another group columns')

    # compute score
    metrics = []
    nan_cols = df.columns[df.isna().any()]
    for col in tqdm(nan_cols):
        nan_rows = df[col].isna()   # select rows that are NaN in this columns
        y_train = df.loc[~nan_rows, col]
        train_pred = stats.loc[~nan_rows, col]
        score = np.sqrt(mean_squared_error(y_train, train_pred))    # RMSE

    metrics.append(score)
    print(f'\n{np.mean(metrics)}')
    return imp

In [15]:
predicted = imp.fillna(imp.mean())      # try cosine/ALS instead?
# predicted = groupstat(imp, gcol=['F_2_2'], func='mean')

In [16]:
aux.save_submission(predicted)

Still contain NaN: False


,row-col,value
0,0-F_1_14,-0.000907
1,0-F_3_23,0.000368
2,1-F_3_24,0.004464
3,2-F_1_2,0.000215
4,2-F_4_2,0.257058


In [ ]:
#